# Preprocesamiento y Búsqueda Semántica — T3 INF239

En este notebook se desarrolla el proceso necesario para trabajar con el corpus de discursos históricos entregado en la tarea. A lo largo del documento se realiza la lectura y preprocesamiento de los textos, se calcula para cada archivo un hash SHA-256 que servirá como identificador único, y se generan sus embeddings utilizando el modelo sentence-transformers. Una vez procesada la información, los documentos completos con su identificador, su texto original y su vector de embedding se insertan en la base de datos MongoDB configurada como Replica Set. Finalmente, se implementa un mecanismo de búsqueda semántica que permite recibir una consulta en lenguaje natural, convertirla en embedding y compararla con todos los documentos mediante la similitud coseno, entregando los resultados más relevantes.

### Importación de librerías

Aquí cargamos las librerías que usaremos durante todo el notebook: conexión a MongoDB, manejo de archivos, hashing, generación de embeddings y cálculo de similitud. Estas importaciones deben ejecutarse antes de cualquier otro paso.

In [39]:
import os
import hashlib
import numpy as np
from pymongo import MongoClient
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

print("Todo importó correctamente ")


Todo importó correctamente 


### Conexión a MongoDB Replica Set

En esta sección se establece la conexión entre el notebook y la base de datos MongoDB configurada como Replica Set. Para ello se utiliza la URI que incluye los tres nodos del clúster y el nombre del set de réplicas. Una vez conectado el cliente, se selecciona la base de datos Politica y la colección Discursos, verificando que la conexión sea exitosa.

In [7]:
from pymongo import MongoClient

# URI del Replica Set (usa los nombres del hosts que agregaste)
uri = "mongodb://mongo1:30001,mongo2:30002,mongo3:30003/?replicaSet=my-replica-set"

client = MongoClient(uri)

db = client["Politica"]
coleccion = db["Discursos"]

print("Conexión exitosa. Colecciones actuales:", db.list_collection_names())


Conexión exitosa. Colecciones actuales: []


### Carga del corpus de discursos
En esta parte se define la ruta donde se encuentran los documentos originales entregados para la Tarea 3. Luego se listan todos los archivos .txt disponibles dentro de la carpeta del corpus. Esto permite verificar que los textos están correctamente ubicados y que se cargaron en la cantidad esperada.

In [8]:
import os

# Ruta donde están los .txt
ruta_discursos = r"C:\T3_BD\T3-202304539k-2023135674\DiscursosOriginales\DiscursosOriginales\DiscursosOriginales"

# Listar los archivos
archivos = [f for f in os.listdir(ruta_discursos) if f.endswith(".txt")]

print("Archivos encontrados:", len(archivos))
for a in archivos:
    print(a)


Archivos encontrados: 680
100020.txt
100033.txt
100172.txt
100178.txt
100227.txt
100260.txt
100296.txt
100302.txt
100385.txt
100422.txt
100454.txt
100509.txt
100547.txt
100603.txt
100990.txt
101030.txt
101041.txt
101075.txt
101103.txt
101169.txt
101255.txt
101299.txt
101310.txt
101337.txt
101383.txt
101415.txt
101452.txt
101556.txt
101614.txt
101688.txt
101766.txt
101951.txt
102005.txt
102024.txt
102066.txt
102200.txt
102228.txt
102280.txt
102351.txt
102437.txt
102450.txt
102498.txt
102541.txt
102687.txt
102784.txt
102791.txt
102858.txt
102960.txt
102987.txt
103043.txt
103122.txt
103207.txt
103226.txt
103263.txt
103313.txt
103359.txt
103407.txt
103470.txt
103511.txt
103651.txt
103681.txt
133807.txt
133835.txt
133961.txt
134115.txt
134150.txt
134204.txt
134278.txt
134376.txt
134637.txt
134658.txt
134740.txt
134801.txt
134896.txt
134933.txt
135058.txt
135074.txt
135110.txt
135207.txt
135221.txt
135347.txt
135384.txt
135449.txt
135513.txt
135653.txt
135757.txt
135814.txt
135819.txt
135842

### Lectura de un documento de ejemplo

Aquí se selecciona uno de los archivos del corpus y se lee su contenido completo.
Esto permite confirmar que los textos están correctamente codificados en UTF-8 y que pueden ser procesados sin problemas. Además, se muestran los primeros caracteres del documento para verificar su estructura.

In [9]:
# Probar leyendo el contenido del primer archivo
primer_archivo = archivos[0]
ruta_completa = os.path.join(ruta_discursos, primer_archivo)

with open(ruta_completa, "r", encoding="utf-8") as f:
    contenido = f.read()

print("Nombre del archivo:", primer_archivo)
print("Primeros 300 caracteres")
print(contenido[:300])


Nombre del archivo: 100020.txt
Primeros 300 caracteres
Muy buenos días:

 

Disfruten de este Sol de invierno. Gabriela Mistral decía que “hay que amar a nuestras ciudades” y agregaba que la belleza de nuestras ciudades nos ennoblece y su fealdad nos envilece. Y es muy importante porque la ciudad es el hogar de todos. Yo conozco países en que cuidan muc


<div style="background-color:#e8f4fd; padding:15px; border-left:5px solid #5ba4e6; border-radius:6px;">
<b>Cálculo del Hash SHA-256</b><br><br>
En esta sección calculamos el hash SHA-256 de cada discurso. Este hash se utiliza como el
identificador único (<code>_id</code>) dentro de MongoDB, asegurando que no se inserten documentos duplicados
y que cada archivo tenga una representación consistente basada en su contenido.<br><br>
Aquí se muestra el cálculo aplicado a un archivo de ejemplo para verificar su formato y longitud.
</div>



In [10]:
import hashlib
import os

# Ruta donde están los discursos
ruta_discursos = r"C:\T3_BD\T3-202304539k-2023135674\DiscursosOriginales\DiscursosOriginales\DiscursosOriginales"

# Archivo para probar
archivo_prueba = archivos[0]
ruta_archivo = os.path.join(ruta_discursos, archivo_prueba)

# Leer contenido
with open(ruta_archivo, "r", encoding="utf-8") as f:
    texto = f.read()

# Generar hash SHA-256
hash_id = hashlib.sha256(texto.encode("utf-8")).hexdigest()

print("Archivo:", archivo_prueba)
print("Hash SHA-256:", hash_id)
print("Largo del hash:", len(hash_id))


Archivo: 100020.txt
Hash SHA-256: e18c554e1328feea02c78d5b43a5866226dbcf22772152f1707dbb115e0da267
Largo del hash: 64


<div style="background-color:#dff6ff; padding:15px; border-radius:8px;">
<b>Validación del cálculo de hash SHA-256</b><br><br>
Para verificar que el cálculo del hash funciona correctamente en todos los archivos del corpus,
probamos el procedimiento con los primeros cinco documentos. Cada archivo debe generar un hash
distinto, siempre de 64 caracteres, representando de manera única su contenido.<br><br>
A continuación se muestran los hashes generados como ejemplo de esta validación.
</div>


In [11]:
# Probar hash para los primeros 5 archivos
hashes_prueba = {}

for nombre_archivo in archivos[:5]:  # los primeros 5
    ruta = os.path.join(ruta_discursos, nombre_archivo)
    
    with open(ruta, "r", encoding="utf-8") as f:
        texto = f.read()
    
    id_hash = hashlib.sha256(texto.encode("utf-8")).hexdigest()
    hashes_prueba[nombre_archivo] = id_hash

# Mostrar resultados
for archivo, h in hashes_prueba.items():
    print(f"{archivo} → {h}")


100020.txt → e18c554e1328feea02c78d5b43a5866226dbcf22772152f1707dbb115e0da267
100033.txt → 5e9c4483f8772eb7494a392b88e91087b5e47448d905d35b96d983ee27974f5d
100172.txt → 4c74324c38c28db64f001afe58645d31e3da2d4c98302cbedefb4131a162b4a2
100178.txt → 32665ed671286af2e9f788476dd7ac4e1662eaa3947865699ed9f8ef27c80743
100227.txt → 2b822ed90a6eed38dc1312374cbeaa6ced40a247b023941fa6266aa140aba485


<div style="background-color:#e6f4ff; padding:12px; border-radius:8px;">
<b>Carga del modelo de embeddings</b><br><br>
Aquí se carga el modelo preentrenado <i>all-MiniLM-L6-v2</i> de <i>sentence-transformers</i>, 
que se utilizará para convertir cada discurso y cada consulta en un vector numérico 
(embedding) dentro de un espacio semántico.
</div>


In [12]:
from sentence_transformers import SentenceTransformer

# Cargar modelo una sola vez
modelo = SentenceTransformer("all-MiniLM-L6-v2")

print("Modelo cargado correctamente")


C:\Users\mpobl\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mpobl\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not insta

Modelo cargado correctamente


<div style="background-color:#e6f4ff; padding:12px; border-radius:8px;">
<b>Generación de embedding para un discurso de ejemplo</b><br><br>
En esta sección se toma uno de los discursos del corpus, se lee su contenido completo 
y se genera su embedding usando el modelo cargado. Luego se muestra la dimensión del 
vector y algunos de sus valores para verificar que el proceso funciona correctamente.
</div>


In [13]:
# Archivo de prueba
archivo_prueba = archivos[0]
ruta_prueba = os.path.join(ruta_discursos, archivo_prueba)

# Leer contenido
with open(ruta_prueba, "r", encoding="utf-8") as f:
    texto = f.read()

# Generar embedding
embedding = modelo.encode(texto)

# Convertir a lista normal para MongoDB
embedding_lista = embedding.tolist()

print("Archivo:", archivo_prueba)
print("Dimensión del embedding:", len(embedding_lista))
print("Primeros 10 valores:", embedding_lista[:10])
 

Archivo: 100020.txt
Dimensión del embedding: 384
Primeros 10 valores: [-0.017531754449009895, -0.0010601116809993982, 0.02968698740005493, -0.0637868195772171, 0.006820754148066044, -0.02250032313168049, 0.020270587876439095, 0.0008010416640900075, 0.01223050057888031, 0.04081955552101135]


<div style="background-color:#d7f3ff; padding:12px; border-radius:8px;">

### Generación de embeddings para múltiples documentos

Una vez verificado que el modelo de *sentence-transformers* funciona correctamente para un documento individual, procedemos a generar los embeddings de varios textos.  
En este paso tomamos los primeros 5 archivos del corpus, calculamos su vector numérico de representación semántica y confirmamos que todos tienen la dimensión esperada (384 valores).  

Este proceso permite validar que el modelo se comporta de forma consistente antes de procesar los 680 documentos completos.

</div>


embeddings_5 = {}

for nombre_archivo in archivos[:5]:
    ruta = os.path.join(ruta_discursos, nombre_archivo)
    
    # leer texto
    with open(ruta, "r", encoding="utf-8") as f:
        texto_archivo = f.read()
    
    # generar embedding
    emb = modelo.encode(texto_archivo).tolist()
    
    # guardar en diccionario
    embeddings_5[nombre_archivo] = emb

# mostrar resumen
for archivo, emb in embeddings_5.items():
    print(f"{archivo} → dimensión {len(emb)}")


<div style="background-color:#d4ecff; padding:14px; border-radius:8px;">

### Procesamiento completo del corpus  

En esta etapa se recorre todo el conjunto de discursos y se realiza el preprocesamiento final.  
Para cada documento se ejecutan tres tareas fundamentales:

1. **Lectura del texto completo** del archivo `.txt`.  
2. **Cálculo del hash SHA-256**, que servirá como identificador único (`_id`) en MongoDB.  
3. **Generación del embedding** mediante el modelo previamente cargado.

Luego, cada documento se organiza dentro de un diccionario con su estructura final y se agrega a una lista que contendrá todo el corpus listo para insertarse en la base de datos.

</div>


In [15]:
import hashlib
import os

documentos_procesados = []

for nombre_archivo in archivos:
    ruta = os.path.join(ruta_discursos, nombre_archivo)

    # leer texto
    with open(ruta, "r", encoding="utf-8") as f:
        texto = f.read()

    # generar hash
    hash_id = hashlib.sha256(texto.encode("utf-8")).hexdigest()

    # generar embedding
    emb = modelo.encode(texto).tolist()

    # construir documento
    documento = {
        "_id": hash_id,
        "texto": texto,
        "embedding": emb
    }

    documentos_procesados.append(documento)

print("Documentos procesados:", len(documentos_procesados))
print("Ejemplo del primero:")
print(documentos_procesados[0].keys())


Documentos procesados: 680
Ejemplo del primero:
dict_keys(['_id', 'texto', 'embedding'])


<div style="background-color:#d0ecff; padding:15px; border-radius:8px;">
<h3 style="margin-top:0;">Inserción de documentos en MongoDB con manejo de duplicados</h3>

En esta sección se insertan los 680 documentos procesados en la colección <strong>Discursos</strong>.  
Al usar el hash SHA-256 como <code>_id</code>, cualquier documento repetido produce un error de clave duplicada.  
Por esto, la inserción se realiza con <code>insert_one()</code> dentro de un bloque <code>try / except</code>, lo que permite:

- Insertar todos los documentos únicos.
- Saltarse los duplicados sin detener el proceso.
- Registrar si ocurre algún error inesperado.

El resultado final muestra cuántos documentos fueron insertados correctamente y cuántos fueron detectados como duplicados.
</div>


In [33]:
insertados = 0
duplicados = 0
errores = 0

for doc in documentos_procesados:
    try:
        coleccion.insert_one(doc)
        insertados += 1
    except Exception as e:
        if "duplicate key" in str(e):
            duplicados += 1
        else:
            errores += 1
            print("Error inesperado:", e)

print("Inserciones exitosas:", insertados)
print("Duplicados saltados:", duplicados)
print("Errores inesperados:", errores)
print("Total documentos procesados:", len(documentos_procesados))


Inserciones exitosas: 679
Duplicados saltados: 1
Errores inesperados: 0
Total documentos procesados: 680


<div style="background-color:#d2ebff; padding:12px; border-radius:8px;">

### Recuperación de documentos desde MongoDB

En este paso recuperamos desde MongoDB todos los discursos previamente insertados.  
Convertimos cada resultado en un diccionario Python para poder procesarlo con normalidad dentro del notebook.  
Esto también nos permite verificar que los embeddings fueron guardados correctamente y que la colección contiene el número esperado de documentos.

</div>


In [36]:
# Recuperar todos los documentos desde MongoDB
import numpy as np

cursor = coleccion.find({}, {"_id": 1, "texto": 1, "embedding": 1})

# Convertir a listas Python manejables
docs_mongo = list(cursor)

print("Documentos cargados desde Mongo:", len(docs_mongo))
print("Ejemplo de claves del primer documento:", docs_mongo[0].keys())
print("Dimensión embedding primer documento:", len(docs_mongo[0]["embedding"]))


Documentos cargados desde Mongo: 679
Ejemplo de claves del primer documento: dict_keys(['_id', 'texto', 'embedding'])
Dimensión embedding primer documento: 384


<div style="background-color:#dbecff; padding:12px; border-radius:8px">

### Función de búsqueda semántica

En esta sección definimos la función responsable de realizar la búsqueda semántica.  
La función recibe una consulta en texto libre, genera su embedding y luego la compara con los embeddings almacenados en MongoDB utilizando **similitud coseno**.  
Finalmente, ordena los resultados desde el más similar hasta el menos similar y retorna los `top_k` documentos más relacionados.

</div>



In [37]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def buscar_discursos(consulta, top_k=5):

    # 1. Generar embedding de la consulta
    embedding_consulta = modelo.encode(consulta)

    resultados = []

    # 2. Comparar la consulta con cada documento
    for doc in docs_mongo:
        emb_doc = np.array(doc["embedding"])

        sim = cosine_similarity(
            embedding_consulta.reshape(1, -1),
            emb_doc.reshape(1, -1)
        )[0][0]

        resultados.append({
            "id": doc["_id"],
            "texto": doc["texto"],
            "similitud": float(sim)
        })

    # 3. Ordenar resultados por similitud descendente
    resultados_ordenados = sorted(
        resultados, key=lambda x: x["similitud"], reverse=True
    )

    return resultados_ordenados[:top_k]

print("Función 'buscar_discursos' creada correctamente.")


Función 'buscar_discursos' creada correctamente.


<div style="background-color:#dbecff; padding:12px; border-radius:8px">

### Prueba de búsqueda semántica

Aquí realizamos una búsqueda de ejemplo para verificar que el sistema funciona correctamente.  
Probamos con la consulta **"educación"** y mostramos los **top 5 discursos más similares**, junto con sus puntuaciones de similitud y un fragmento inicial del texto.

</div>


In [38]:
consulta = "educación"
resultados = buscar_discursos(consulta)

print("Consulta:", consulta)
print("Top 5 documentos más similares:\n")

for i, r in enumerate(resultados, start=1):
    print(f"{i}. ID: {r['id']} — Similitud: {r['similitud']:.4f}")
    print("Primeros 200 caracteres del texto:")
    print(r["texto"][:200])
    print("-" * 80)


Consulta: educación
Top 5 documentos más similares:

1. ID: a6458b4fa6d85ad7d370dc28cf0f9425d5b4f4724447d39ab95a781bd5145801 — Similitud: 0.6373
Primeros 200 caracteres del texto:
Muy buenos días:

 

Quiero saludar con mucho cariño a la ministra de Educación, al presidente de la FIDE, a los directivos, pero principalmente a todos ustedes, que son los que durante muchas décadas
--------------------------------------------------------------------------------
2. ID: bb00be2446947a4a70657c382f860e1b6286696452ae13388887f9ec9af539f3 — Similitud: 0.5620
Primeros 200 caracteres del texto:
Muy buenos días:

 

Bienvenidos a La Moneda. Y quiero decirles que ésta es una reunión que tiene una gran significancia y una gran proyección para nosotros, porque no voy a insistir en lo importante 
--------------------------------------------------------------------------------
3. ID: 88b6dd7083a5cef0ff86fd829bfc36e06002dca9b04108aa7522ae3579f1aecb — Similitud: 0.5596
Primeros 200 caracteres del texto:
Mu

In [3]:
from pymongo import MongoClient

uri = "mongodb://mongo1:30001,mongo2:30002,mongo3:30003/?replicaSet=my-replica-set"
client = MongoClient(uri)

db = client["Politica"]
coleccion = db["Discursos"]


In [8]:
import json

# Obtener todos los documentos de la colección
cursor = coleccion.find({}, {"_id": 1, "texto": 1, "embedding": 1})

# Convertir cursor a lista compatible con JSON
exportar = []
for doc in cursor:
    exportar.append({
        "_id": doc["_id"],
        "texto": doc["texto"],
        "embedding": doc["embedding"]
    })

# Nombre final del archivo con sus roles
nombre_archivo = "discursos-final-202304539k-2023135674.json"

# Guardar en el sistema
with open(nombre_archivo, "w", encoding="utf-8") as f:
    json.dump(exportar, f, ensure_ascii=False, indent=2)

print("Archivo JSON exportado correctamente:", nombre_archivo)


ServerSelectionTimeoutError: mongo1:30001: [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms),mongo3:30003: [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms),mongo2:30002: [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 691a3b15e3941c161aba934d, topology_type: ReplicaSetNoPrimary, servers: [<ServerDescription ('mongo1', 30001) server_type: Unknown, rtt: None, error=AutoReconnect('mongo1:30001: [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dich